## .NET异步编程

### 常见概念

<font color="orange">已经有多线程了，为何还要异步</font>
- 多线程与异步是不同的概念
    - 异步并不意味着多线程，单线程同样可以异步
    - 异步默认借助线程池
    - 多线程经常阻塞，而异步要求不阻塞
- 多线程与异步的适用场景不同
    - 多线程
        - 适合CPU密集型操作
        - 适合长期运行的任务
        - 线程的创建与销毁开销较大
        - 提供更底层的控制，操作线程，锁，信号量等
        - 线程不易于传参及返回
        - 线程的代码书写较为繁琐
    - 异步
        - 适合IO密集型操作
        - 适合短暂的小任务
        - 避免线程阻塞，提高系统响应能力

<font color="orange">什么是异步任务（Task）</font>
- 包含了异步任务的各种状态的一个引用类型
    - 正在运行，完成，结果，报错等
    - 另有ValueTask值类型版本
- 对于异步任务的抽象
    - 开启异步任务后，当前线程并不会阻塞，而是可以去做其他事情
    - 异步任务默认会借助线程池在其他线程上运行
    - 获取结果后回到之前的状态
- 任务的结果
    - 返回值为Task的方法表示异步任务没有返回值
    - 返回值为Task<T>则表示返回值类型为T的Task

In [2]:
using System.Threading;
var task = new Task<string>(() => 
{
    Thread.Sleep(1500);
    return "done";
});

task.Status.Display();
task.Start();
task.Status.Display();
Thread.Sleep(1000);
task.Status.Display();
Thread.Sleep(2000);
task.Status.Display();
task.Result.Display();

Created

WaitingToRun

Running

RanToCompletion

done

<font color="orange">异步方法</font>
- 将方法标记为async后，可以在方法内使用await关键字
- await关键字会等待异步任务的结束并获得结果
- async + await会将方法包装成状态机，await类似于检查点 ———— MoveNext方法会被底层调用，从而切换状态
- async Task
    - 返回值依旧是Task类型，但是在其中可以使用await关键字
    - 在其中写返回值可以直接写Task<T>中的T类型，不用包装成Task<T>
- async void 
    - 同样是状态机，但缺少记录状态的Task对象
    - 无法聚合异常（Aggregate Exception），需要谨慎处理异常
    - 几乎只用于对于事件的注册
- 异步编程具有传染性
    - 一处async，处处async
    - 几乎所有自带方法都提供了异步的版本
    
    

In [7]:
using System.Runtime.CompilerServices;

Helper.PrintThreadId("Before");
await FooAsync();
Helper.PrintThreadId("After");

async Task FooAsync()
{
    Helper.PrintThreadId("Before");
    await Task.Delay(1000);
    Helper.PrintThreadId("After");
}

class Helper
{
    private static int index = 1;
    public static void PrintThreadId(string? message = null, [CallerMemberName] string? name = null)
    {
        var title = $"{index}: {name}";
        if(!string.IsNullOrEmpty(message))
            title += $"@ {message}";
        title.Display();
        Environment.CurrentManagedThreadId.Display();
        Interlocked.Increment(ref index);
    }
}

1: <Initialize>@ Before

10

2: FooAsync@ Before

10

3: FooAsync@ After

53

4: <Initialize>@ After

53

<font color="orange">重要思想：不阻塞！</font>
- await会暂时释放当前线程，使得线程可以执行其他工作，而不必阻塞线程直到异步操作完成
- 不要在异步方法里用任何方式阻塞当前线程
- Task.Wait() & Task.Result
    - 如果任务没有完成，则会阻塞当前线程，容易导致死锁 
    - Task.GetAwaiter().GetResult()  —— —— 不会将Exception包装为AggregateException
- Task.Delay() vs. Thread.Sleep()
    - 后者会阻塞当前的线程，这与异步编程的理念不符
    - 前者是一个异步任务，会立刻释放当前的线程
- IO等操作的同步方法
- 其他繁重且耗时的任务

<font color="orange">同步上下文</font>
- 一种管理和协调线程的机制，允许开发者将代码的执行切换到特定的线程
- WinForms与WPF拥有同步上下文（UI线程），而控制台程序默认没有
- ConfigureAwait(false)
    - 配置任务通过await方法结束后是否会到原来的线程，默认为true
    - 一般只有UI线程会采用这种策略
- TaskScheduler
    - 控制Task的调度方式和运行线程
        - 线程池线程Default
        - 当前线程CurrentThread
        - 单线程上下文STAThread
        - 长时间运行线程LongRunning
    - 优先级，上下文，执行状态等
    

<font color="orange">一发既忘（Fire-and-forget）</font>
- 调用一个异步方法、但是并不使用await或阻塞的方式去等待它的结束
- 无法观察任务的状态（是否完成、是否报错等）


### 简单任务

<font color="green">如何创建异步任务</font>
- Task.Run()
- Task.Factory.StartNew()
    - 提供了更多功能，比如TaskCreationOpentions.LongRunning
    - Task.Run相当于简化版
- new Task + Task.Start()

In [10]:
var inputs = Enumerable.Range(1,10).ToArray();

var tasks = new List<Task<int>>();

foreach(var input in inputs)
{
    tasks.Add(HeavyJob(input));
}
await Task.WhenAll(tasks); //防止阻塞主线程

var outputs = tasks.Select(_ => _.Result).ToArray();
outputs.DisplayTable();

async Task<int> HeavyJob(int input)
{
    await Task.Delay(1000);
    return input * input;
}

value
1
4
9
16
25
36
49
64
81
100


<font color="green">任务的取消</font>
- CancellationTokenSource + CancellationToken
- OperationCanceledException & TaskCanceledException
- 推荐异步方法都带上CancellationToken这一传参，可以不用，但不能没有

In [13]:
var cts = new CancellationTokenSource();
try
{
    var task = Task.Delay(10000,cts.Token);
    Thread.Sleep(2000);
    cts.Cancel();

    await task;
}
catch(TaskCanceledException ex)
{
    "TaskCanceled".Display();
}
finally
{
    cts.Dispose();
}

TaskCanceled

### 常见误区

<font color="yellow">异步一定是多线程？</font>
- 异步编程不是必须多线程来实现 —— 时间片轮转
- 比如可以在单个线程上使用异步IO或事件驱动的编程模型（EAP）
- 单线程异步：自己定好计时器，到时间之前先去做别的事情
- 多线程异步: 将任务交给不同的线程，并由自己来进行指挥调度


<font color="yellow">异步方法一定要写成async Task？</font>
- async 关键字只是用来配合await使用，从而将方法包装为状态机
- 本质上仍然是Task，只不过提供了语法糖，并且函数体中可以直接return Task的泛型类型
- 接口中无法声明async Task

<font color="yellow">await一定会切换同步上下文？</font>
- 在使用await关键字调用并等待一个异步任务时，异步方法不一定会立刻来到新的线程上
- 如果await了一个已经完成的任务（包括Task.Delay(0)），会直接获得结果

<font color="yellow">异步可以全面取代多线程？</font>
- 异步编程与多线程有一定的关系，但两者并不是可以完全相互取代

<font color="yellow">Task.Result一定会阻塞当前线程？</font>
- 如果任务已经完成，那么Task.Result可以直接得到结果

<font color="yellow">开启的异步任务一定不会阻塞当前线程？</font>
- 若异步方法里增加了异步方法，则会阻塞当前线程

### 异步编程的同步机制

In [16]:
var inputs = Enumerable.Range(1,10).ToArray();
var sam = new SemaphoreSlim(2,2); //通过Semaphore实现同步
var tasks = new List<Task<int>>();

foreach(var input in inputs)
{
    tasks.Add(HeavyJob(input));
}
await Task.WhenAll(tasks); //防止阻塞主线程

var outputs = tasks.Select(_ => _.Result).ToArray();
outputs.DisplayTable();

async Task<int> HeavyJob(int input)
{
    await sam.WaitAsync(); //等待一个通行证
    await Task.Delay(1000);
    sam.Release();
    return input * input;
}

value
1
4
9
16
25
36
49
64
81
100
